# ResNet50 categorical model implementation:
## Import packages:

In [17]:
import os 
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime
import glob
import shutil
import matplotlib.pyplot as plt

## Activate GPU:

In [18]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


## Set script parameters:

In [19]:
# Model:
model_name = "ResNet50"
model_type = "Categorical"

# Intro:
# 1. Set current working directory:
os.chdir("D:/GitHub/DeepNeuralNetworksRepoPy")
# 2. Create 'ResNet50' folder in current workind directory:
if os.path.exists(os.path.join(os.getcwd(), model_name)) == False:
    os.mkdir(os.path.join(os.getcwd(), model_name))
# 3. Create 'Categorical' subfolder in 'ResNet50' main folder:
if os.path.exists(os.path.join(os.getcwd(), model_name, model_type)) == False:
    os.mkdir(os.path.join(os.getcwd(), model_name, model_type))

## Directories:

In [20]:
# train_dir = "D:/GitHub/Datasets/Cifar10/train"
train_dir = "D:/GitHub/Datasets/Cifar10_2/train"
# validation_dir = "D:/GitHub/Datasets/Cifar10/validation"
validation_dir = "D:/GitHub/Datasets/Cifar10_2/validation"
test_dir = "D:/GitHub/Datasets/Cifar10/test"
models_store_dir = os.path.join(os.getcwd(), model_name, model_type)
models_repo_store_dir = "D:/GitHub/DeepNeuralNetworksRepoPy_Models_Store"
callback_model_checkpoint_path = os.path.join(models_store_dir, "keras_model.weights.{epoch:02d}-{val_accuracy:.4f}-{val_loss:.4f}.hdf5")
callback_tensorboard_path = os.path.join(models_store_dir, "logs")
callback_csv_logger_path = os.path.join(models_store_dir, "_".join([datetime.datetime.now().strftime("%d-%m-%Y %H-%M-%S"), model_name, "model_optimization_logger.csv"]))

## Clear session:

In [21]:
tf.keras.backend.clear_session()

# Model
## Setting pipeline parameters values:

In [22]:
# Image:
image_size = 224
channels = 3

# Model structure:
weights = "imagenet"
include_top = False
activation = "softmax"

# Model compilation:
loss = "categorical_crossentropy"
optimizer = "adam"
metrics = ["accuracy"]

# Training:
batch_size = 16
class_mode = "categorical"
shuffle = True
epochs = 3
early_stopping_patience = 10
reduce_lr_on_plateu_patience = 5
monitor = "val_accuracy"
save_best_only = True
verbose = 1
write_graph = False
write_grads = False
write_images = False
restore_best_weights = False
histogram_freq = 1
min_delta = 0

## ResNet50 model architecture:

In [23]:
input_tensor = tf.keras.applications.ResNet50(
    include_top = include_top,
    weights = weights,
    input_shape = (image_size, image_size, channels))

output_tensor = input_tensor.output
output_tensor = tf.keras.layers.GlobalAveragePooling2D()(output_tensor)
output_tensor = tf.keras.layers.Dense(units = len(os.listdir(train_dir)), activation = activation)(output_tensor)

model = tf.keras.models.Model(
    inputs = input_tensor.input, 
    outputs = output_tensor)

## Visualize model:

In [24]:
model.summary()
tf.keras.utils.plot_model(model)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

## Model compilation:

In [25]:
model.compile(
    loss = loss,
    optimizer = optimizer,
    metrics = metrics)

## Generators:

In [26]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center = False,
    samplewise_center = False,
    featurewise_std_normalization = False,
    samplewise_std_normalization = False,
    zca_whitening = False,
    zca_epsilon = 1e-06,
    rotation_range = 0,
    width_shift_range = 0,
    height_shift_range = 0,
    brightness_range = None,
    shear_range = 0,
    zoom_range = 0,
    channel_shift_range = 0,
    fill_mode = "nearest",
    cval = 0,
    horizontal_flip = False,
    vertical_flip = False,
    rescale = 1/255,
    preprocessing_function = None,
    data_format = None,
    validation_split = 0, 
    dtype = None)

train_generator = train_datagen.flow_from_directory(
    directory = train_dir,
    target_size = (image_size, image_size),
    batch_size = batch_size,
    class_mode = class_mode,
    classes = os.listdir(train_dir),
    shuffle = shuffle)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255)
validation_generator = validation_datagen.flow_from_directory(
    directory = validation_dir,
    target_size = (image_size, image_size),
    batch_size = batch_size,
    class_mode = class_mode,
    classes = os.listdir(train_dir),
    shuffle = shuffle)

Found 2000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


## Model optimization:

In [27]:
history = model.fit_generator(
    generator = train_generator,
    steps_per_epoch = np.ceil(train_generator.n/train_generator.batch_size),
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = np.ceil(validation_generator.n/validation_generator.batch_size),
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath = callback_model_checkpoint_path,
            monitor = monitor,
            verbose = verbose,
            save_best_only = save_best_only),
        tf.keras.callbacks.EarlyStopping(
            monitor = monitor,
            min_delta = min_delta,
            verbose = verbose,
            patience = early_stopping_patience,
            restore_best_weights = restore_best_weights),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor = monitor,
            factor = 0.1,
            patience = reduce_lr_on_plateu_patience,
            verbose = verbose),
        tf.keras.callbacks.TensorBoard(
            log_dir = callback_tensorboard_path,
            histogram_freq = histogram_freq,
            write_graph = write_graph,
            write_grads = write_grads,
            write_images = write_images),
        tf.keras.callbacks.CSVLogger(
            filename = callback_csv_logger_path,
            separator = ";",
            append = True)])

Epoch 1/3
124/125 [============================>.] - ETA: 0s - loss: 2.3844 - accuracy: 0.2787
Epoch 00001: val_accuracy improved from -inf to 0.10000, saving model to D:\GitHub\DeepNeuralNetworksRepoPy\ResNet50\Categorical\keras_model.weights.01-0.1000-6.0539.hdf5
125/125 [==============================] - 38s 308ms/step - loss: 2.3822 - accuracy: 0.2770 - val_loss: 6.0539 - val_accuracy: 0.1000
Epoch 2/3
124/125 [============================>.] - ETA: 0s - loss: 1.6358 - accuracy: 0.4057
Epoch 00002: val_accuracy did not improve from 0.10000
125/125 [==============================] - 35s 282ms/step - loss: 1.6345 - accuracy: 0.4065 - val_loss: 3.2028 - val_accuracy: 0.1000
Epoch 3/3
124/125 [============================>.] - ETA: 0s - loss: 1.3009 - accuracy: 0.5363
Epoch 00003: val_accuracy improved from 0.10000 to 0.10100, saving model to D:\GitHub\DeepNeuralNetworksRepoPy\ResNet50\Categorical\keras_model.weights.03-0.1010-3.6705.hdf5
125/125 [==============================] - 37s 

In [28]:
history_df = pd.DataFrame(history.history)
history_df["epoch"] = np.arange(len(history_df)) + 1
history_df

,loss,accuracy,val_loss,val_accuracy,lr,epoch
0,2.382173,0.2770,6.053885,0.100,0.001,1
1,1.634453,0.4065,3.202785,0.100,0.001,2
2,1.297337,0.5375,3.670465,0.101,0.001,3


## Remove not optimal models:

In [29]:
os.chdir(models_store_dir)
saved_models = glob.glob(os.path.join(models_store_dir, "*.hdf5"))
if len(saved_models) > 1:
    for i in np.arange(len(saved_models))[:-1]:
        print("Remove .hdf5 file:", saved_models[i])
        os.remove(saved_models[i])

Remove .hdf5 file: D:\GitHub\DeepNeuralNetworksRepoPy\ResNet50\Categorical\keras_model.weights.01-0.1000-6.0539.hdf5


## Remove logs folders:

In [30]:
logs_folders = glob.glob(os.path.join(models_store_dir, "logs"))
for i in np.arange(len(logs_folders)):
    print("Remove logs folder:", logs_folders[i])
    shutil.rmtree(logs_folders[i], ignore_errors=True)

Remove logs folder: D:\GitHub\DeepNeuralNetworksRepoPy\ResNet50\Categorical\logs


## Save optimal model in local models repository:

In [31]:
optimal_model = glob.glob(os.path.join(models_store_dir, "*.hdf5"))[0]
optimal_model_repo_dir = os.path.join(models_repo_store_dir, "_".join([model_type, model_name, "Model.hdf5"]))
shutil.copyfile(src = optimal_model,
               dst = optimal_model_repo_dir)
print("Optimal model directory:", optimal_model_repo_dir)
os.remove(optimal_model)

Optimal model directory: D:/GitHub/DeepNeuralNetworksRepoPy_Models_Store\Categorical_ResNet50_Model.hdf5


# Validate optimized model:
## Clear session and import the best trained model:

In [32]:
os.chdir(models_store_dir)
tf.keras.backend.clear_session()
optimal_model_repo_dir = os.path.join(models_repo_store_dir, "_".join([model_type, model_name, "Model.hdf5"]))
model = tf.keras.models.load_model(optimal_model_repo_dir, compile = False)
model.compile(
    loss = loss,
    optimizer = optimizer,
    metrics = metrics)

## Visualize model:

In [33]:
model.summary()
tf.keras.utils.plot_model(model)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

## Generators:

In [34]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255)
train_generator = train_datagen.flow_from_directory(
    directory = train_dir,
    target_size = (image_size, image_size),
    batch_size = batch_size,
    class_mode = class_mode,
    classes = os.listdir(train_dir),
    shuffle = False)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255)
validation_generator = validation_datagen.flow_from_directory(
    directory = validation_dir,
    target_size = (image_size, image_size),
    batch_size = batch_size,
    class_mode = class_mode,
    classes = os.listdir(train_dir),
    shuffle = False)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255)
test_generator = test_datagen.flow_from_directory(
    directory = test_dir,
    target_size = (image_size, image_size),
    batch_size = batch_size,
    class_mode = class_mode,
    shuffle = False)

Found 2000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


## Evaluate generators:

In [35]:
print("Evaluate train generator:")
train_evaluation = model.evaluate_generator(
    generator = train_generator, 
    steps = np.ceil(train_generator.n/train_generator.batch_size))
print("Train dataset:", "loss =", np.round(train_evaluation[0], 4), "accuracy =", np.round(train_evaluation[1], 4), "\n")

print("Evaluate validation generator:")
validation_evaluation = model.evaluate_generator(
    generator = validation_generator, 
    steps = np.ceil(validation_generator.n/validation_generator.batch_size))
print("Validation dataset:", "loss =", np.round(validation_evaluation[0], 4), "accuracy =", np.round(validation_evaluation[1], 4), "\n")

print("Evaluate test generator:")
test_evaluation = model.evaluate_generator(
    generator = test_generator, 
    steps = np.ceil(test_generator.n/test_generator.batch_size))
print("Test dataset:", "loss =", np.round(test_evaluation[0], 4), "accuracy =", np.round(test_evaluation[1], 4), "\n")

Evaluate train generator:
Train dataset: loss = 3.6583 accuracy = 0.1005 

Evaluate validation generator:
Validation dataset: loss = 3.6837 accuracy = 0.101 

Evaluate test generator:
Test dataset: loss = 3.6837 accuracy = 0.101 



## Predict generators:

In [36]:
print("Predict train generator:")
train_probabilites = model.predict_generator(
    generator = train_generator,
    steps = np.ceil(train_generator.n/train_generator.batch_size),
    verbose = 1)

print("Predict validation generator:")
validation_probabilites = model.predict_generator(
    generator = validation_generator,
    steps = np.ceil(validation_generator.n/validation_generator.batch_size),
    verbose = 1)

print("Predict test generator:")
test_probabilites = model.predict_generator(
    generator = test_generator,
    steps = np.ceil(test_generator.n/test_generator.batch_size),
    verbose = 1)

Predict train generator:
125/125 [==============================] - 6s 46ms/step
Predict validation generator:
63/63 [==============================] - 3s 46ms/step
Predict test generator:
63/63 [==============================] - 3s 46ms/step


## Save predictions:

In [37]:
train_results = pd.DataFrame(train_probabilites, columns = os.listdir(train_dir))
train_results["Actual_Class"] = train_generator.classes
train_results["Filepath"] = train_generator.filepaths
train_results["Model_Name"] = model_name
train_results = train_results[["Filepath"] + os.listdir(train_dir) + ["Actual_Class"] + ["Model_Name"]]
train_predictions_filename = os.path.join(models_store_dir, "_".join([datetime.datetime.now().strftime("%d-%m-%Y %H-%M-%S"), model_name, "train_binary_predictions.csv"]))
train_results.to_csv(train_predictions_filename)
print("Train predictions saved:", train_predictions_filename, "\n")

validation_results = pd.DataFrame(validation_probabilites, columns = os.listdir(validation_dir))
validation_results["Actual_Class"] = validation_generator.classes
validation_results["Filepath"] = validation_generator.filepaths
validation_results["Model_Name"] = model_name
validation_results = validation_results[["Filepath"] + os.listdir(validation_dir) + ["Actual_Class"] + ["Model_Name"]]
validation_predictions_filename = os.path.join(models_store_dir, "_".join([datetime.datetime.now().strftime("%d-%m-%Y %H-%M-%S"), model_name, "validation_binary_predictions.csv"]))
validation_results.to_csv(validation_predictions_filename)
print("validation predictions saved:", validation_predictions_filename, "\n")

test_results = pd.DataFrame(test_probabilites, columns = os.listdir(test_dir))
test_results["Actual_Class"] = test_generator.classes
test_results["Filepath"] = test_generator.filepaths
test_results["Model_Name"] = model_name
test_results = test_results[["Filepath"] + os.listdir(test_dir) + ["Actual_Class"] + ["Model_Name"]]
test_predictions_filename = os.path.join(models_store_dir, "_".join([datetime.datetime.now().strftime("%d-%m-%Y %H-%M-%S"), model_name, "test_binary_predictions.csv"]))
test_results.to_csv(test_predictions_filename)
print("test predictions saved:", test_predictions_filename, "\n")

Train predictions saved: D:\GitHub\DeepNeuralNetworksRepoPy\ResNet50\Categorical\31-05-2020 18-17-00_ResNet50_train_binary_predictions.csv 

validation predictions saved: D:\GitHub\DeepNeuralNetworksRepoPy\ResNet50\Categorical\31-05-2020 18-17-00_ResNet50_validation_binary_predictions.csv 

test predictions saved: D:\GitHub\DeepNeuralNetworksRepoPy\ResNet50\Categorical\31-05-2020 18-17-00_ResNet50_test_binary_predictions.csv 

